In [1]:
using Random
using Distributions

Simulation for the implementation of hamming error correction code

In [2]:
# this function calculates how many parity bits will be needed

function red_bits(x)
    for i in 1:length(x)
        if 2^i >= length(x) + i + 1
            return i
        end
    end
end

red_bits (generic function with 1 method)

In [3]:
# this function determines the location of the data bits withing the codeword

function pos_red(data,red)
    a = 0
    b = 1
    c = length(data)
    res = zeros(c+red)
    res =round.(Int, res)
    data = reverse(data)
    # 7+4+1 = 12
    for i in 1:(c+red+1)
        if(i == 2^a )# a<red) # 1 2 4 8 16 32
            #res = res * "0"
            a+=1
        else #3 5 6 7 9 10 11 12
            if (b <= c)
                res[i] = data[b]
                b += 1
            end
        end
    end
    return reverse(res)
end

    

pos_red (generic function with 1 method)

In [4]:
function par(data,red)
    R1 = (1,3)
    u = Int(length(data))
    out = zeros(u)
    rdata = reverse(data)
    for i in 1:red
        a=0
        for j in 1:u
            if (j & 2^i) == 2^i
                a= a^rdata[j]
            end
        end
        
        b=u-2^i
        print(i)
        println(b)
       # if(b>0)
        #    out[:b] = data[:b]
        #end
        
    end
            
    return out


end

par (generic function with 1 method)

In [5]:
function par2(data,red)
    n = length(data)
    rdata = reverse(data)
    sum = 0
    R1 = (1,3,5,7,9,11,13,15) # R1 = 1, 3, 5, 7, 9, 11
    R2 = (2,3,6,7,10,11,14,15)# 2,3,6,7,10,11
    R4 = (4,5,6,7,12,13,14,15) #4, 5, 6, 7
    R8 = (8,9,10,11,12,13,14,15) # 8,9,10,11
    for i in R1
        if(i<=n)
            sum += rdata[i]
        end
    end
    sum = sum%2
    rdata[1] = sum
    sum = 0
    for i in R2
        if(i<=n)
            sum += rdata[i]
        end
    end
    rdata[2] = sum%2
    sum = 0
    for i in R4
        if(i<=n)
            sum += rdata[i]
        end
    end
    rdata[4] = sum%2
    sum = 0
    for i in R8
        if(i<=n)
            sum += rdata[i]
        end
    end
    rdata[8] = sum%2
    sum = 0
    
    return reverse(rdata)
    
end


par2 (generic function with 1 method)

In [6]:
function err_detect(data,u)
    data_size = length(data)
    pos = 0
    rdata = reverse(data)
    for i in 1:u
        value = 0
        for j in 1:data_size
            if (j & 2^i) == 2^i
                value = value^rdata[j]
            end
        end
        
        pos = pos + value*(10^i)
    end
    
    return pos
    
end


err_detect (generic function with 1 method)

In [7]:
function err_detect1(data,u)
    n=length(data)
    R1 = (1,3,5,7,9,11,13,15) # R1 = 1, 3, 5, 7, 9, 11
    R2 = (2,3,6,7,10,11,14,15)# 2,3,6,7,10,11
    R4 = (4,5,6,7,12,13,14,15) #4, 5, 6, 7
    R8 = (8,9,10,11,12,13,14,15) # 8,9,10,11
    rdata = reverse(data)
    err = 0
    sum = 0
    for i in R1
        if(i<=n)
            sum += rdata[i]
        end
    end
    err = sum%2
    sum = 0
    for i in R2
        if(i<=n)
            sum += rdata[i]
        end
    end
    err += (sum%2)*2
    sum = 0
    for i in R4
        if(i<=n)
            sum += rdata[i]
        end
    end
    err += (sum%2)*4
    sum = 0
    for i in R8
        if(i<=n)
            sum += rdata[i]
        end
    end
    err += (sum%2)*8
    return err
    
end

err_detect1 (generic function with 1 method)

In [8]:
@show red_bits([1,1,1,1,1,1])
@show pos_red([1,0,1,1,0,0,1],4)
@show par2([1,0,1,0,1,0,0,0,1,0,0],4)
# only use 7 bits

red_bits([1, 1, 1, 1, 1, 1]) = 4
pos_red([1, 0, 1, 1, 0, 0, 1], 4) = [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
par2([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0], 4) = [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0]


11-element Vector{Int64}:
 1
 0
 1
 0
 1
 0
 0
 1
 1
 1
 0

In [17]:

err_detect1([1,0,1,0,1,1,0,1,1,1,0],4)

6

above is code for calculating the number of parity bits, the location of the parity bits, the value of the parity bits as well as detecting errors in the data. Simulation of the performance will be done below


In [10]:
rng = MersenneTwister(1234)
# mutiple of 7,use 7*15 105 for now
input = bitrand(rng,105)
@show input

input = Bool[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1]


105-element BitVector:
 0
 0
 1
 1
 0
 1
 1
 1
 0
 0
 0
 0
 1
 ⋮
 0
 1
 0
 1
 0
 1
 1
 1
 1
 0
 1
 1

In [11]:
# for every 7 bits 4 parity bits will be added so new array is (7+4)*15 =165
# 1 8 15 22

encoded = zeros(165)
cal = zeros(11)
for i in 1:15
    b = pos_red(input[7*(i-1)+1:7*(i-1)+7],4)
    encoded[11*(i-1)+1:11*(i-1)+11] = par2(b,4)
end

@show encoded

encoded = [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]


165-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 1.0

In [12]:
sent = zeros(length(encoded))
for i in 1:length(encoded)
    if encoded[i] == 0
        sent[i] = -1
    else
        sent[i] = 1
    end
end

@show sent

sent = [-1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0]


165-element Vector{Float64}:
 -1.0
 -1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
 -1.0
  1.0
 -1.0
 -1.0
  1.0
 -1.0
  ⋮
 -1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
 -1.0
  1.0
 -1.0
  1.0

In [25]:
# now add noise to the sent signal
#this isnt working too nicely
b = rand(Uniform(-1,1),length(encoded))
SNR = -4
sigma = 1/sqrt(10^(SNR/10)*2)
#make a term for the snr
b = b*sigma
sent_noise = sent+b

@show sent_noise

sent_noise = [-2.0968221003896703, -1.2286194664506993, 0.600047664093601, 1.5326614042669076, 0.7399687348451398, -1.2605014633977272, 0.7289802478488282, -0.6907287914386218, 1.1486652111830375, -1.5907585324855615, -1.855054131262436, 2.0375254184158242, -1.47297957391258, -0.6145267334972613, 1.7427301499228394, -0.19416303594946782, -0.3516190743621168, 2.0110351798557, 2.009056182364784, -0.796320841064431, 0.8263314768259631, 0.025432176189893685, 0.09132198211588327, -2.055836174532099, -1.6942454218221275, 0.30463613908363163, 1.3044658214469251, 1.35558808585762, -0.5211187152286696, -0.33077475414076296, 0.7031181844081997, -1.174568135669408, 0.17416753367252213, -0.42360900496837695, 0.417606012213694, 0.1643790602377454, 0.11657251385002776, -0.38436219055902265, -0.24718016531960385, 1.154846595536425, 1.040761339482525, 1.5150650862897743, -1.9079719751830333, 0.4060505351479218, -0.09369320048544183, 0.4887589151838463, -0.6461394749543079, -1.6664812267441196, 1.94312

165-element Vector{Float64}:
 -2.0968221003896703
 -1.2286194664506993
  0.600047664093601
  1.5326614042669076
  0.7399687348451398
 -1.2605014633977272
  0.7289802478488282
 -0.6907287914386218
  1.1486652111830375
 -1.5907585324855615
 -1.855054131262436
  2.0375254184158242
 -1.47297957391258
  ⋮
 -0.22131207663925978
  0.7331001646223527
 -0.0046898601363820624
  1.3030842552219102
  0.9678078056493903
  1.4898059165456876
 -0.460328944520464
  1.552983119815899
 -0.71559755417577
  1.3012914583904929
 -0.8762783791585773
 -0.06863930101577309

In [26]:
# now demod and make a decision
rec = zeros(length(sent_noise))

for i in 1:length(sent_noise)
    if sent_noise[i]>0
        rec[i] = 1
    else
        rec[i] = 0
    end
end

@show encoded
@show rec

encoded = [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
rec = [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.

165-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 0.0
 1.0
 0.0
 1.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0

In [27]:
#calcuting the difference between the encoded signal and the demod signal 
count = 0
for i in 1:length(rec)
    if rec[i]!=encoded[i]
        count+=1
    end
end

print("there are ",count," errors")

there are 12 errors

In [49]:
# now lets apply the detection algorithm to see how many errors are detected then correct thos errors and see
# actual amount of errors afterwords
n = length(rec)
corrected = rec
for i in 1:15
    b = err_detect1(rec[11*(i-1)+1:11*(i-1)+11],4)
    print(Int(b)," ")
end
println()
print(rec[100:110])
print("-", encoded[100:110])

0 1 0 8 11 11 4 4 0 0 0 3 9 1 11 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]-[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]